In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from fuzzywuzzy import process

username = 'postgres'   
password = 'floox2024!'   
hostname = 'localhost'   
port     = '5432'  
db = 'netfloox' 
 
conn_string = f"host={hostname} dbname={db} user={username} password={password}  "
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

/Users/john/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/john/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
cursor.execute(''' select tb.tconst, tb."primaryTitle", tb."titleType", tb.genres, 
tb."startYear", tb."runtimeMinutes", 
array_to_string(cbt."actor",','),
array_to_string(cbt."director",','),
array_to_string(cbt."writer",','),
tr."averageRating", tr."numVotes",
count(ta.title) As "countTitleByRegion"
from "titleBasics" tb 
full outer join "titleAkas" ta 
on tb.tconst = ta."titleId" 
full outer join cast_by_title cbt 
on tb.tconst = cbt.tconst 
full outer join "titleRatings" tr 
on tb.tconst  = tr.tconst 
where
tb."isAdult" = 0  AND tb."titleType" LIKE 'movie'
group by tb.tconst,
cbt."actor",cbt."director",cbt."writer",
tr."averageRating", tr."numVotes"
ORDER BY tr."numVotes" DESC NULLS LAST, tr."averageRating" DESC NULLS LAST
LIMIT 500000 ''')

movie = cursor.fetchall()
df = pd.DataFrame(movie , columns=["tconst","title","titleType","genres","startYear","runtimeMinutes","actor","director","writer","averageRating","numVotes","countTitleByRegion" ])

In [4]:
df

,tconst,title,titleType,genres,startYear,runtimeMinutes,actor,director,writer,averageRating,numVotes,countTitleByRegion
0,tt0111161,The Shawshank Redemption,movie,Drama,1994.0,142,"Tim Robbins,Morgan Freeman,Bob Gunton,William ...",Frank Darabont,Stephen King,9.3,2851572,82
1,tt0468569,The Dark Knight,movie,"Action,Crime,Drama",2008.0,152,"Christian Bale,Heath Ledger,Aaron Eckhart,Mich...",Christopher Nolan,"Jonathan Nolan,David S. Goyer,Bob Kane",9.0,2832886,77
2,tt1375666,Inception,movie,"Action,Adventure,Sci-Fi",2010.0,148,"Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot ...",Christopher Nolan,None,8.8,2515111,67
3,tt0137523,Fight Club,movie,Drama,1999.0,139,"Edward Norton,Meat Loaf,Zach Grenier,Brad Pitt",David Fincher,"Chuck Palahniuk,Jim Uhls",8.8,2286960,61
4,tt0944947,Game of Thrones,tvSeries,"Action,Adventure,Drama",2011.0,4189,None,None,None,9.2,2250465,69
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,tt0260558,Whatever Happened to Susan Jane?,movie,"Adventure,Drama",1982.0,60,"Ann Block,Francesca Rosa,Lulu,Greg Cruikshank,...",Marc Huestis,None,7.8,49,2
499996,tt0262167,To louna park,tvSeries,"Comedy,Drama",1974.0,60,None,None,None,7.8,49,2
499997,tt0283239,Valley of the Dolls,tvEpisode,Documentary,2001.0,None,None,None,None,7.8,49,1
499998,tt0289994,Losing Hope,movie,Comedy,2001.0,80,"LG Taylor,Chip Hajel,Andy Sims,Jamie McCoy",Tony Urban,Genie Davis,7.8,49,3


knn model 


In [6]:

df['title'].fillna('', inplace=True)
df['genres'] = df['genres'].fillna('')
df['genres'] =  df['genres'].str.replace(',', ' ')
df['actor'] = df['actor'].astype(str) 
df['director'] =df['director'].astype(str)
df['writer'] =df['writer'].astype(str)
df['genres'] = df['genres'].fillna('')
df['title'] = df['title'].fillna('')
df['averageRating'] =df['averageRating'].astype(str)

/var/folders/w2/c305x1jx409c6wqclptt2rfr0000gn/T/ipykernel_52991/3623176088.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['title'].fillna('', inplace=True)


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
# For simplicity,  fill missing values for categorical data with a placeholder and numerical with median
df.fillna({'title': 'Unknown', 'director': 'Unknown', 'actor': 'Unknown', 
           'genres': 'Unknown', 'writer': 'Unknown'}, inplace=True)

# Imputing missing numerical values with median
num_imputer = SimpleImputer(strategy='median')
df[['startYear', 'runtimeMinutes']] = num_imputer.fit_transform(df[['startYear', 'runtimeMinutes']])

# Normalize 'startYear' and 'runtimeMinutes'
scaler = MinMaxScaler()
df[['startYear', 'runtimeMinutes']] = scaler.fit_transform(df[['startYear', 'runtimeMinutes']])


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Combine textual features
df['combined_features'] = df['title']+ ' ' + df['title']+ ' ' + df['director'] + ' ' + df['actor'] + ' ' + df['genres'] + ' ' + df['writer']

# TF-IDF Vectorization for the combined textual features
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
combined_features_tfidf = vectorizer.fit_transform(df['combined_features'])


In [52]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
combined_features_tfidf = vectorizer.fit_transform(df['combined_features'])
def suggest_film_name_contains(input_film_name, df):
    # Normalize the input film name to lowercase for case-insensitive matching
    input_film_name_lower = input_film_name.lower()
    
    # Filter the DataFrame for titles that contain the input string
    contains_match_df = df[df['title'].str.lower().str.contains(input_film_name_lower)]
    
    # If one or more matches are found, return the title of the first match
    if not contains_match_df.empty:
        return contains_match_df.iloc[0]['title']  # Return the first matching title
    else:
        return None
    
# Building the recommendation model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(combined_features_tfidf)

def get_recommendations(film_name, df, model_knn, vectorizer):
    # Transform film_name to match the features space
    film_features = vectorizer.transform([film_name])
    
    # Find similar movies
    distances, indices = model_knn.kneighbors(film_features, n_neighbors=50)
    
    recommendations = []
    
    for i in range(0, len(distances.flatten())):

        if i == 0:
            print(f'Recommendations for "{film_name}":\n')
        else:
            # Append recommendation details to the list
            index = indices.flatten()[i]  # Get the original DataFrame index for the recommended item
            recommendations.append({
                'index': df.index[index],
                'title': df.iloc[indices.flatten()[i]]['title'],
                'Distance': distances.flatten()[i],
                'rating': df['averageRating'][i] 
            })
           
            
    
    # Convert the list of recommendations to a DataFrame
 
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df 



film_name1 = input("Enter the name of the film: ")
film_name = suggest_film_name_contains(film_name1, df)
rec_knn = get_recommendations(film_name, df, model_knn, vectorizer)
rec_knn


Recommendations for "Batman v Superman: Dawn of Justice":



,index,title,Distance,rating
0,422422,Superman 5,0.453957,9.0
1,189911,Rifftrax: Batman v. Superman,0.493414,8.8
2,407114,They Call Me Superman,0.525379,8.8
3,463732,Batman Beyond,0.547010,8.8
4,249269,We Are Superman,0.553181,8.9
5,442939,Superman: Agent of Batman,0.555744,8.7
6,495071,Batman: The Dawn Sequel,0.571314,8.7
7,433256,Justice,0.576727,9.2
8,433812,Justice,0.581133,8.9
9,405332,Batman: Most Wanted,0.588762,9.0


cosine smiu

In [42]:
#Import count from scikit-learn

from sklearn.feature_extraction.text import CountVectorizer
dfc=df
dfc['startYear'] = dfc['startYear'].astype(str)
dfc['runtimeMinutes'] = dfc['runtimeMinutes'].astype(str)
dfc['combined_features'] = dfc['title']+ ' ' + dfc['title'] + ' ' + dfc['director'] + ' ' + dfc['actor'] + ' ' + dfc['genres'] + ' ' + dfc['writer'] + ' ' + dfc['startYear'] + ' ' + dfc['runtimeMinutes']
couv = CountVectorizer(stop_words='english', max_features= 500)
count_matrix = couv.fit_transform(dfc['combined_features'])
count_matrix

<500000x500 sparse matrix of type '<class 'numpy.int64'>'
	with 2726537 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
# Map film names to indices
film_to_index = pd.Series(df.index, index=df['title']).to_dict()

# Function to get cosine similarity for a given film name
def get_cosine_similarity(film_name, count_matrix, film_to_index):
    # Get the index of the film from the name
    idx = film_to_index.get(film_name)
    if idx is not None:
        # Compute cosine similarity between the film vector and all vectors
        cosine_sim = cosine_similarity(count_matrix, count_matrix[idx].reshape(1, -1))
        return cosine_sim
    else:
        return None
cosine_sim = get_cosine_similarity(film_name, count_matrix, film_to_index)

if cosine_sim is not None:
    print(cosine_sim)
else:
    print("Film not found.")


[[0.        ]
 [0.15569979]
 [0.40201513]
 ...
 [0.        ]
 [0.        ]
 [0.07106691]]


In [50]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Create a mapping from film titles to their indices in the feature matrix
film_to_index = pd.Series(df.index, index=df['title']).to_dict()

def get_cosine_sim_recommendations(film_name, df, combined_features_tfidf, film_to_index):
    # Check if the film exists in our mapping
    if film_name in film_to_index:
        # Get the index of the film from its name
        idx = film_to_index[film_name]
        # Compute cosine similarity between the film's features and the features of all films
        cosine_sim = cosine_similarity(combined_features_tfidf, combined_features_tfidf[idx].reshape(1, -1))
        
        # Get pairwise similarity scores for all films with that film
        sim_scores = list(enumerate(cosine_sim.flatten()))
        
        # Sort the films based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the scores of the 50 most similar films, skip the first one since it is the query film itself
        sim_scores = sim_scores[1:51]
        
        # Get the film indices
        film_indices = [i[0] for i in sim_scores]
        
        # Return the top 50 most similar films
        return  df.iloc[film_indices][['title']].reset_index()
    else:
        return None

# Example usage
recommendations = get_cosine_sim_recommendations(film_name, df, combined_features_tfidf, film_to_index)
if recommendations is not None:
    print("Top recommendations:")
    print(recommendations)
else:
    print("Film not found.")


Top recommendations:
     index                                title
0       42               Avengers: Infinity War
1       80              Avengers: Age of Ultron
2      107           Captain America: Civil War
3       22                         The Avengers
4      973                         The Gray Man
5    83749                        Alien Endgame
6   411475                              Endgame
7       88  Captain America: The Winter Soldier
8    61445                              Endgame
9   442869                          The EndGame
10  441107                       Vegas Avengers
11   23102                              Endgame
12   76490                         Endgame 2050
13  267450                           An Endgame
14  378571                     Spanish Avengers
15  490844         Endgame (Miller's Reckoning)
16  310236                The People's Avengers
17  123865                       Three Avengers
18   10505                   Operation: Endgame
19   69742         

hybirde model 

In [51]:
content_cosine = pd.DataFrame(recommendations).astype(str)
rec_knn=rec_knn.astype(str)
matched_df = pd.merge(content_cosine, rec_knn, on=('index', 'title'), how='inner')
matched_df.head(5)

,index,title,Distance,rating
0,42,Avengers: Infinity War,0.6943197276083837,8.0
1,80,Avengers: Age of Ultron,0.7168560569000819,8.1
2,22,The Avengers,0.6051319458456506,8.8
3,83749,Alien Endgame,0.4228199961451088,9.0
4,411475,Endgame,0.4300435381913107,8.8


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

couv = CountVectorizer(stop_words='english', max_features= 5000)
count_matrix = couv.fit_transform(dfc['combined_features'])

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix)


def find_all_close_titles(tit, df):
    # Lowercase the title for case-insensitive matching
    tit = tit.lower()
    # Ensure the title column is in lowercase for matching
    temp_df = df.copy()
    temp_df['title'] = temp_df['title'].str.lower()
    # Find titles that exactly match or closely match the partial title
    close_matches = temp_df[temp_df['title'].str.contains(tit)]
    return close_matches

def rec_mov(partial_movie_title, df, cosine_sim):
    partial_movie_title = partial_movie_title.lower()
    close_matches_df = find_all_close_titles(partial_movie_title, df)
    
    if close_matches_df.empty:
        return pd.DataFrame()  # Return an empty DataFrame if no matches
    
    # Assuming there could be multiple close matches, we'll use the first one for simplicity
    first_match_index = close_matches_df.index[0]
    sim_scores = list(enumerate(cosine_sim[first_match_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Get top 5 matches, excluding the movie itself
    
    movie_indices = [i[0] for i in sim_scores]
    recommended_movies = df.iloc[movie_indices]
    
    return recommended_movies[['title', 'actor']]  # Adjust columns to those actually available

# Assuming you have a user input mechanism in place
user_input_movie = input("Enter the film's name: ")
recommended_movies = rec_mov(user_input_movie, df, cosine_sim).head(5)
if not recommended_movies.empty:
    print(recommended_movies)
else:
    print("No recommendations found.")


                         title  \
217        The Matrix Reloaded   
279     The Matrix Revolutions   
776   The Matrix Resurrections   
512                Cloud Atlas   
1221         Jupiter Ascending   

                                                  actor  
217   Keanu Reeves,Laurence Fishburne,Carrie-Anne Mo...  
279   Keanu Reeves,Laurence Fishburne,Carrie-Anne Mo...  
776   Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mate...  
512       Tom Hanks,Halle Berry,Hugh Grant,Hugo Weaving  
1221  Channing Tatum,Mila Kunis,Eddie Redmayne,Sean ...  
